<a href="https://colab.research.google.com/github/tsuli/LSC_project/blob/main/Task2_v05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# specify the data file name and url
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00240/'
datafile = url + 'UCI%20HAR%20Dataset.zip'

# download the zip file from the web server using curl
!curl $datafile --output UCI_HAR_Dataset.zip

# unzip the file
!unzip -qq UCI_HAR_Dataset.zip

# change the directory name to remove spaces
!mv -f UCI\ HAR\ Dataset UCI_HAR_DATASET

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 58.1M  100 58.1M    0     0  13.4M      0  0:00:04  0:00:04 --:--:-- 13.4M


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [3]:
# load the features and labels (subtract 1 as the labels aren't indexed from 0)
ytest = np.loadtxt('UCI_HAR_DATASET/test/y_test.txt').astype(np.float32) -1
ytrain = np.loadtxt('UCI_HAR_DATASET/train/y_train.txt').astype(np.float32) -1

# load the x,y,z body accelerations test data
xx=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/body_acc_x_test.txt').astype(np.float32) 
yy=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/body_acc_y_test.txt').astype(np.float32) 
zz=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/body_acc_z_test.txt').astype(np.float32) 

# concatenate the arrays along the last dimension
xtest = np.concatenate((xx[:,:,None],yy[:,:,None],zz[:,:,None]),axis=2)
# (using None here adds an extra dimension of size 1 to the end of the array)

# follow the same approach for the train data
xx=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/body_acc_x_train.txt').astype(np.float32) 
yy=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/body_acc_y_train.txt').astype(np.float32) 
zz=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/body_acc_z_train.txt').astype(np.float32) 

xtrain = np.concatenate((xx[:,:,None],yy[:,:,None],zz[:,:,None]),axis=2)

xtrain, xvalid, ytrain, yvalid = train_test_split(xtrain, ytrain, test_size=0.2, random_state=42)
ytrain_enc = tf.keras.utils.to_categorical(ytrain)
yvalid_enc = tf.keras.utils.to_categorical(yvalid)
ytest_enc = tf.keras.utils.to_categorical(ytest)
xtrain = xtrain.astype(np.float32)
xvalid = xvalid.astype(np.float32)
xtest = xtest.astype(np.float32)

print(xtrain.shape, ytrain_enc.shape, xvalid.shape, yvalid_enc.shape, xtest.shape, ytest_enc.shape)

(5881, 128, 3) (5881, 6) (1471, 128, 3) (1471, 6) (2947, 128, 3) (2947, 6)


In [4]:
input_layer = tf.keras.Input(shape=(128,3))
output_layer1 = tf.keras.layers.Conv1D(filters=32, kernel_size=(4), padding = 'same')(input_layer)
output_layer2 = tf.keras.layers.BatchNormalization()(output_layer1)
output_layer3 = tf.keras.layers.ReLU()(output_layer2)
output_layer4 = tf.keras.layers.GlobalAveragePooling1D()(output_layer3)
output_layer5 = tf.keras.layers.Dense(units=6,activation=tf.nn.softmax)(output_layer4)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer5)

In [5]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)]

In [6]:
model.fit(xtrain,  ytrain_enc, 
          epochs=100,
          batch_size=64, 
          callbacks=callbacks,
          validation_data=(xvalid, yvalid_enc))

Epoch 1/100
92/92 [==============================] - 2s 11ms/step - loss: 1.5478 - accuracy: 0.3896 - val_loss: 1.7168 - val_accuracy: 0.2848
Epoch 2/100
92/92 [==============================] - 1s 10ms/step - loss: 1.3343 - accuracy: 0.5375 - val_loss: 1.6149 - val_accuracy: 0.3243
Epoch 3/100
92/92 [==============================] - 1s 9ms/step - loss: 1.2019 - accuracy: 0.5540 - val_loss: 1.4593 - val_accuracy: 0.4432
Epoch 4/100
92/92 [==============================] - 1s 9ms/step - loss: 1.0920 - accuracy: 0.5724 - val_loss: 1.2498 - val_accuracy: 0.5751
Epoch 5/100
92/92 [==============================] - 1s 10ms/step - loss: 1.0113 - accuracy: 0.6077 - val_loss: 1.0858 - val_accuracy: 0.4908
Epoch 6/100
92/92 [==============================] - 1s 10ms/step - loss: 0.9540 - accuracy: 0.6359 - val_loss: 0.9666 - val_accuracy: 0.6111
Epoch 7/100
92/92 [==============================] - 1s 9ms/step - loss: 0.9110 - accuracy: 0.6586 - val_loss: 0.9004 - val_accuracy: 0.5914
Epoch 8/1

In [35]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 128, 3)]          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 128, 32)           416       
_________________________________________________________________
batch_normalization_5 (Batch (None, 128, 32)           128       
_________________________________________________________________
re_lu_5 (ReLU)               (None, 128, 32)           0         
_________________________________________________________________
average_pooling1d_3 (Average (None, 64, 32)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 64, 6)             198       
Total params: 742
Trainable params: 678
Non-trainable params: 64
____________________________________________________________

1, s 4ms/step - loss: 0.5741 - accuracy: 0.7747 - val_loss: 0.5855 - val_accuracy: 0.7485
epochs=100, batch_size=256

s 10ms/step - loss: 0.6661 - accuracy: 0.7368 - val_loss: 0.6702 - val_accuracy: 0.6621, epochs=100, batch_size=64